# L6: Long Context Prompting

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import libraries

In [ ]:
from ai21 import AI21Client
from ai21.models.chat import ChatMessage

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

## Load API key and create AI21Client

In [ ]:
from utils import get_ai21_api_key
ai21_api_key = get_ai21_api_key()

client = AI21Client(api_key=ai21_api_key)

In [ ]:
Filing2023 = open('Nvidia_10K_20230129.txt', 'r', encoding='utf-8').read()
Filing2024 = open('Nvidia_10K_20240128.txt', 'r', encoding='utf-8').read()

In [ ]:
system = "You are a SEC Filing assistant."
separator="\n\n***************\n\n"

prompt = f"""Summarize the following SEC annual report: {Filing2024}{separator}
Summary:"""

messages = [ChatMessage(role='system', content=system),
            ChatMessage(role='user', content=prompt)]

chat_completions = client.chat.completions.create(
    messages=messages,
    model="jamba-1.5-large",
    max_tokens=2000,
    temperature=0.3)

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [ ]:
chat_completions.choices[0].message.content

In [ ]:
def jamba_chat(query,filing, model="jamba-1.5-large", max_tokens=2000, temperature=0.3):

    system_message="""You are a SEC Filing assistant.
    Answer questions or performance requested tasks based on provided SEC filing doc in the prompt."""
    separator="\n\n***************\n\n"

    prompt = f"""Use the following SEC filing: {filing}{separator}
    Answer the following question based on the context above: {query}"""

    messages = [
        ChatMessage(role="system", content=system_message),
        ChatMessage(role="user", content=prompt)
    ]

    chat_completions = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature
    )

    return chat_completions.choices[0].message.content

In [ ]:
JAMBA_QUERY = "What drove the revenue growth in 2024?"

response = jamba_chat(JAMBA_QUERY, Filing2024)
print(response)

In [ ]:
def jamba_json(query,filing, model="jamba-1.5-large", max_tokens=2000, temperature=0):

    system_message="""You are a SEC Filing assistant.
    Answer questions in JSON format based on provided SEC filing doc in the prompt."""
    separator="\n\n***************\n\n"

    prompt = f"""Use the following SEC filing: {filing}{separator}
    Generate JSON output for Answer the following question based on the context above: {query}"""

    messages = [
        ChatMessage(role="system", content=system_message),
        ChatMessage(role="user", content=prompt)
    ]

    chat_completions = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        response_format={"type": "json_object"}
    )

    return chat_completions.choices[0].message.content

In [ ]:
JAMBA_QUERY = """Create a JSON output of the financial performance from this 10K filing,
including fiscal year end date, revenue, gross profit, 
net income per share, revenue by segment, 
revenue by geo region."""

response = jamba_json(JAMBA_QUERY, Filing2024)
print(response)

In [ ]:
all_filings = Filing2023 + "\n\n||||||\n\n" + Filing2024

JAMBA_QUERY = """Create an html table of the financial performance 
from both of the filings, including the fiscal year, 
revenue, and earnings per share for year 2023 and 2024."""

response = jamba_chat(JAMBA_QUERY, all_filings)

from IPython.display import display, Markdown, HTML
display(HTML(response))